In [3]:
import logging
from abc import ABC, abstractmethod
from datetime import datetime
import re
import sys

logging.basicConfig(filename="HospitalLogs.log", level=logging.INFO, format="%(asctime)s : %(levelname)s : %(message)s")
class BaseHospital(ABC):
    def __init__(self, user_name):
        '''
        Defined Logger and creates log file as the user enters User Name 
        '''
        self.user_name = user_name
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        log_file = f"{self.user_name}.log"

        self.logger = logging.getLogger(self.user_name)
        self.logger.setLevel(logging.INFO)

        file_handler = logging.FileHandler(log_file)
        formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
        file_handler.setFormatter(formatter)

        self.logger.addHandler(file_handler)

    @abstractmethod
    def input_data(self):
        pass

class Doctor(BaseHospital): 
    def __init__(self, user_name):
        super().__init__(user_name)
        self.license_no = None
        self.name = None
        self.phone = None
        self.email = None

    def input_data(self):
        self.license_no = self.DocLicenseNo()
        self.name = self.DocName()
        self.phone = self.DocPhone()
        self.email = self.DocEmail()

    def DocLicenseNo(self):
      ls_no = input("Enter License No Format: NNXXXXNN: ").strip()
      try:
        pattern = "^[0-9]{2}[A-Z]{4}[0-9]{2}$"
        if re.match(pattern, ls_no):
            self.logger.info(f"License No saved: {ls_no}")
            return ls_no
        else:
            raise Exception("Invalid License No")
      except Exception:
            self.logger.error("Invalid License No")
            print(sys.exc_info()[1])
            return None

    def DocName(self):
     name = input("Enter First Name and Last Name: ").strip().title()
     try:
        pattern = "^[A-Z][a-z]* [A-Z][a-z]*$"
        if re.match(pattern, name):
            self.logger.info(f"Doctor name saved: {name}")
            return name
        else:
            raise Exception("Invalid name format")
     except Exception:
            self.logger.error("Invalid name format")
            print(sys.exc_info()[1])
            return None

    def DocPhone(self):
      phone = input("Enter Phone number (10 digits): ").strip()
      try:
        if len(phone) == 10 and phone.isdigit():
            self.logger.info(f"Phone number saved: {phone}")
            return phone
        else:
            raise Exception("Invalid phone number")
      except Exception:
            self.logger.error("Invalid phone number")
            print(sys.exc_info()[1])
            return None

    def DocEmail(self):
      email = input("Enter Email: ").strip()
      try:
        pattern = r"^[A-Za-z0-9._%+-]+@[a-zA-Z0-9.-]+\.[A-Za-z]{2,}$"
        if re.match(pattern, email):
            self.logger.info(f"Email saved: {email}")
            return email
        else:
            raise Exception("Invalid email format")
      except Exception:
            self.logger.error("Invalid email format")
            print(sys.exc_info()[1])
            return None
    

class OPD(BaseHospital):
    def __init__(self, user_name):
        super().__init__(user_name)
        self.opd_id = None
        self.visiting_date = None
        self.visiting_time = None
        self.medicine_id = None
        self.patient_id = None
        self.doc_ls_no = None

    def input_data(self):
        self.visiting_date = self.visitingDate()
        self.visiting_time = self.visitingTime()

    def visitingDate(self):
        vst_date = input("Enter Visiting Date (YYYY-MM-DD): ")
        try:
            visiting_date = datetime.strptime(vst_date, "%Y-%m-%d")
            if visiting_date <= datetime.today():
                self.logger.info("Visiting Date saved: %s", visiting_date)
                return visiting_date
            else:
                raise Exception()
        except Exception:
            self.logger.error("Visiting date cannot be in the future.")
            print(sys.exc_info()[1])
            return None
        
    def visitingTime(self):
        try:
            visiting_time = datetime.now().strftime("%H:%M:%S")
            self.logger.info("Visiting Time saved: %s", visiting_time)
            return visiting_time
        except Exception as e:
            self.logger.error(e)
            print(sys.exc_info()[1])
            return None
    

class IPD(BaseHospital):
    def __init__(self, user_name):
        super().__init__(user_name)
        self.admission_id = None
        self.admission_date = None
        self.discharge_date = None
        self.medicine_id = None
        self.patient_id = None
        self.doc_ls_no = None

    def input_data(self):
        self.admission_date = self.admissionDate()
        self.discharge_date = self.dischargeDate()

    def admissionDate(self):
        admn_date = input("Enter Admission Date (YYYY-MM-DD): ").strip()
        try:
            admission_date = datetime.strptime(admn_date, "%Y-%m-%d")
            if admission_date <= datetime.today():
                self.logger.info("Admission date saved: %s", admission_date)
                return admission_date
            else:
                raise Exception()
        except Exception:
            self.logger.error("Admission date cannot be in the future")
            print(sys.exc_info()[1])
            return None

    def dischargeDate(self):
        disch_date = input("Enter Discharge Date (YYYY-MM-DD): ").strip()
        try:
            discharge_date = datetime.strptime(disch_date, "%Y-%m-%d")
            if (
                self.admission_date
                and discharge_date <= datetime.today()
                and discharge_date >= self.admission_date
            ):
                self.logger.info("Discharge date saved: %s", discharge_date)
                return discharge_date
            else:
                raise Exception()
        except Exception:
            self.logger.error("Discharge date cannot be in the future and must be after the admission date")
            print(sys.exc_info()[1])
            return None
        
class Patient(BaseHospital):
    def __init__(self, user_name):
        super().__init__(user_name)
        self.id = None
        self.name = None
        self.gender = None
        self.dob = None
        self.phone = None
        self.age = None
        self.dept = None

    def input_data(self):
        self.name = self.patientName()
        self.gender = self.patientGender()
        self.dob = self.patientdob()
        self.phone = self.patientPhone()
        self.dept = self.chooseDepartment()

    def patientName(self):
        name = input("Enter Patient Name: ").strip()
        try:
            if all(char.isalpha() or char.isspace() for char in name):
                self.logger.info(f"Patient name saved: {name}")
                return name
            else:
                raise Exception()
        except Exception:
            self.logger.error("Only alphabetical characters and spaces are allowed")
            print(sys.exc_info()[1])
            return None

    def patientGender(self):
        gender = int(input("Enter 1 for Male, 2 for Female, 3 for Others: "))
        try:
            gender_map = {1: "Male", 2: "Female", 3: "Others"}
            if gender in gender_map:
                self.logger.info(f"Gender saved: {gender_map[gender]}")
                return gender_map[gender]
            else:
                raise Exception()
        except Exception:
            self.logger.error("Invalid gender selection")
            print(sys.exc_info()[1])
            return None

    def patientdob(self):
        dob = input("Enter Date of Birth (YYYY-MM-DD): ")
        try:
            dob_date = datetime.strptime(dob, "%Y-%m-%d")
            if dob_date <= datetime.today():
                self.age = datetime.today().year - dob_date.year
                self.logger.info(f"DOB saved: {dob}, Age: {self.age}")
                return dob_date.strftime("%Y-%m-%d")
            else:
                raise Exception()
        except Exception:
            self.logger.error("DOB cannot be in the future")
            print(sys.exc_info()[1])
            return None

    def patientPhone(self):
      phone = input("Enter Phone number (10 digits): ").strip()
      try:
        if len(phone) == 10 and phone.isdigit():
            self.logger.info(f"Phone number saved: {phone}")
            return phone
        else:
            raise Exception()
      except Exception:
            self.logger.error("Invalid phone number")
            print(sys.exc_info()[1])
            return None

    def chooseDepartment(self):
      department = input("Are you admitted to OPD or IPD, Enter 'OPD' or 'IPD': ").strip().upper()
      try: 
        if department == "OPD":
            opd = OPD(self.user_name)
            opd.input_data()
        elif department == "IPD":
            ipd = IPD(self.user_name)
            ipd.input_data()
        else:
            raise Exception()
      except Exception:
            print("Invalid department. Please try again.")


In [4]:
d=Doctor('revati')
d.input_data()

In [5]:
e=Patient('Raju')
e.input_data()